In [69]:
# IMPORTANTO BIBLIOTECAS
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime
from plyer import notification 
import requests
from tqdm import tqdm

In [93]:

# Criando função de alerta
def alerta(api, erro):
    notification.notify(
        title='ATENÇÃO: Falha ao acessar API',
        message=(f'Falha ao acessar a API {api}. \nErro: {erro}. \n{datetime.now()}.'),        
        timeout=5)

# Chamando a função dentro do bloco try-except
try:
    df_habilidades_full = get_base_habilidades()
    salva_bd(df_habilidades_full, "habilidades")
except Exception as e:
    alerta(api="habilidades", erro=e)

  0%|          | 0/541 [00:00<?, ?it/s]


In [94]:
# ACESSANDO APIS VALIDANDO COM A FUNÇÃO ALERTA

pokemons = requests.get("https://pokeapi.co/api/v2/pokemon")
alerta("'pokemons'", pokemons.status_code)
pokemons_habilidades = requests.get("https://pokeapi.co/api/v2/ability")
alerta("'pokemons_habilidades'", pokemons_habilidades.status_code)
taxa_evolucao = requests.get("https://pokeapi.co/api/v2/growth-rate")
alerta("'taxa_evolucao'", taxa_evolucao.status_code)

In [96]:
def tabelas_bd():
        ##Retorna um dataframe com as tabelas
    
    conn = sqlite3.connect('pokeapi.db')

    query = "SELECT name FROM sqlite_master WHERE type='table'"
    schema = pd.read_sql_query(query, conn)

    conn.close()

    return schema
def salva_bd(df, nome_tabela):
    '''
        Salva dataframe df na tabela nome_tabela.
    '''
    conn = sqlite3.connect('pokeapi.db')

    # Escrever o DataFrame na tabela 'nome_tabela'
    df.to_sql(nome_tabela, conn, if_exists='replace', index=False)

    conn.close()

    return True
def carrega_bd(nome_tabela):
    '''
        Carrega tabela nome_tabela num dataframe. 
    '''
    conn = sqlite3.connect('pokeapi.db')

    # Executar uma consulta SELECT na tabela 'produtos' e converter em um DataFrame
    query = f"SELECT * FROM {nome_tabela}"
    df = pd.read_sql(query, conn)

    conn.close()

    return df

In [97]:
def get_json_api(url):
    '''
        Request GET url e retorna json de saida
    '''

    response = requests.get(url)

    if response.status_code == 200:
        data_json = response.json()
        return data_json
    
    # Erro
    raise Exception(f"erro request,  {response.status_code} - {url}\n{response.text}")

In [98]:
pokemons.json()

{'count': 1302,
 'next': 'https://pokeapi.co/api/v2/pokemon?offset=20&limit=20',
 'previous': None,
 'results': [{'name': 'bulbasaur',
   'url': 'https://pokeapi.co/api/v2/pokemon/1/'},
  {'name': 'ivysaur', 'url': 'https://pokeapi.co/api/v2/pokemon/2/'},
  {'name': 'venusaur', 'url': 'https://pokeapi.co/api/v2/pokemon/3/'},
  {'name': 'charmander', 'url': 'https://pokeapi.co/api/v2/pokemon/4/'},
  {'name': 'charmeleon', 'url': 'https://pokeapi.co/api/v2/pokemon/5/'},
  {'name': 'charizard', 'url': 'https://pokeapi.co/api/v2/pokemon/6/'},
  {'name': 'squirtle', 'url': 'https://pokeapi.co/api/v2/pokemon/7/'},
  {'name': 'wartortle', 'url': 'https://pokeapi.co/api/v2/pokemon/8/'},
  {'name': 'blastoise', 'url': 'https://pokeapi.co/api/v2/pokemon/9/'},
  {'name': 'caterpie', 'url': 'https://pokeapi.co/api/v2/pokemon/10/'},
  {'name': 'metapod', 'url': 'https://pokeapi.co/api/v2/pokemon/11/'},
  {'name': 'butterfree', 'url': 'https://pokeapi.co/api/v2/pokemon/12/'},
  {'name': 'weedle', 'u

In [99]:
def get_base_pokemons_url():
    
        ##request completo iterativo da tabela pokemons_url
        ##retorna dataframe da tabela 
    
    next_url = "https://pokeapi.co/api/v2/pokemon"
    df_pokemons_full = pd.DataFrame()
    while next_url is not None:
        print(next_url, end='\r')
        
        # GET 
        json_data = get_json_api(next_url)

        #para dataframe
        df_pokemons = pd.DataFrame(json_data['results'])

        #append/concat  
        df_pokemons_full = pd.concat([df_pokemons_full,df_pokemons],ignore_index=True)

        #next page
        next_url = json_data['next']
    
    return df_pokemons_full

In [100]:
pokemons_habilidades.json()

{'count': 367,
 'next': 'https://pokeapi.co/api/v2/ability?offset=20&limit=20',
 'previous': None,
 'results': [{'name': 'stench', 'url': 'https://pokeapi.co/api/v2/ability/1/'},
  {'name': 'drizzle', 'url': 'https://pokeapi.co/api/v2/ability/2/'},
  {'name': 'speed-boost', 'url': 'https://pokeapi.co/api/v2/ability/3/'},
  {'name': 'battle-armor', 'url': 'https://pokeapi.co/api/v2/ability/4/'},
  {'name': 'sturdy', 'url': 'https://pokeapi.co/api/v2/ability/5/'},
  {'name': 'damp', 'url': 'https://pokeapi.co/api/v2/ability/6/'},
  {'name': 'limber', 'url': 'https://pokeapi.co/api/v2/ability/7/'},
  {'name': 'sand-veil', 'url': 'https://pokeapi.co/api/v2/ability/8/'},
  {'name': 'static', 'url': 'https://pokeapi.co/api/v2/ability/9/'},
  {'name': 'volt-absorb', 'url': 'https://pokeapi.co/api/v2/ability/10/'},
  {'name': 'water-absorb', 'url': 'https://pokeapi.co/api/v2/ability/11/'},
  {'name': 'oblivious', 'url': 'https://pokeapi.co/api/v2/ability/12/'},
  {'name': 'cloud-nine', 'url': 

In [101]:
def get_base_habilidades_url():
    
        ##tabela habilidades_url
        ##retorna dataframe da tabela 
    
    next_url = "https://pokeapi.co/api/v2/ability"
    df_habilidades_full = pd.DataFrame()
    while next_url is not None:
        print(next_url, end='\r')
        
        # GET 
        json_data = get_json_api(next_url)

        #para dataframe
        df_habilidade = pd.DataFrame(json_data['results'])

        #append/concat  
        df_habilidades_full = pd.concat([df_habilidades_full,df_habilidade],ignore_index=True)

        #next page
        next_url = json_data['next']
    
    return df_habilidades_full

In [102]:
taxa_evolucao.json()

{'count': 6,
 'next': None,
 'previous': None,
 'results': [{'name': 'slow',
   'url': 'https://pokeapi.co/api/v2/growth-rate/1/'},
  {'name': 'medium', 'url': 'https://pokeapi.co/api/v2/growth-rate/2/'},
  {'name': 'fast', 'url': 'https://pokeapi.co/api/v2/growth-rate/3/'},
  {'name': 'medium-slow', 'url': 'https://pokeapi.co/api/v2/growth-rate/4/'},
  {'name': 'slow-then-very-fast',
   'url': 'https://pokeapi.co/api/v2/growth-rate/5/'},
  {'name': 'fast-then-very-slow',
   'url': 'https://pokeapi.co/api/v2/growth-rate/6/'}]}

In [103]:
def get_base_evolucao_url():
    
        ##Tabela evolucao_url
        ##retorna dataframe da tabela 
    
    next_url = "https://pokeapi.co/api/v2/evolution-chain"
    df_evolucao_full = pd.DataFrame()
    while next_url is not None:
        print(next_url, end='\r')
        
        # GET 
        json_data = get_json_api(next_url)

        #para dataframe
        df_evolucao = pd.DataFrame(json_data['results'])

        #append/concat  
        df_evolucao_full = pd.concat([df_evolucao_full,df_evolucao],ignore_index=True)

        #next page
        next_url = json_data['next']
    
    return df_evolucao_full

In [104]:
def get_base_pokemons():

    df_pokemons_url = carrega_bd("pokemons_url")

    df_pokemons_full = pd.DataFrame()

    for url in tqdm(df_pokemons_url['url'].values):

        #GET
        json_data = get_json_api(url)

        #para dataframe e append
        id              = json_data['id']
        name            = json_data['name']
        abilities       = json_data['abilities']
        height          = json_data['height']
        weight          = json_data['weight']
        base_experience = json_data['base_experience']

        sr_pokemon = pd.Series({
            "url": url,
            "pokemon_id": id,
            "name": name,
            "height": height,
            "weight": weight,
            "base_experience": base_experience,
            "habilidades": str(abilities)
        })

        #append/concat 
        df_pokemons_full = pd.concat([df_pokemons_full,sr_pokemon.to_frame().T],ignore_index=True)
        
    return df_pokemons_full

In [105]:
def get_base_habilidades():

    df_habilidades_url = carrega_bd("habilidades_url")

    df_habilidades_full = pd.DataFrame()

    for url in tqdm(df_habilidades_url['url'].values):

        #GET
        json_data = get_json_api(url)

        #para dataframe e append
        id             = json_data['id']
        name           = json_data['name']
        generation     = json_data['generation']['name']
        is_main_series = json_data['is_main_series']
        effect         = "\n---\n".join([effect['effect'] for effect in json_data['effect_entries']])

        sr_habilidade = pd.Series({
            "url": url,
            "ability_id":id,
            "name": name,
            "generation": generation,
            "is_main_series": is_main_series,
            "effect": effect
        })

        #append/concat 
        df_habilidades_full = pd.concat([df_habilidades_full,sr_habilidade.to_frame().T],ignore_index=True)

    return df_habilidades_full

In [106]:
def get_base_evolucao():

    df_evolucao_url = carrega_bd("evolucao_url")

    df_evolucao_full = pd.DataFrame()

    for url in tqdm(df_evolucao_url['url'].values):

        #GET
        json_data = get_json_api(url)

        #para dataframe e append
        id             = json_data['id']
        name           = json_data['name']
        generation     = json_data['generation']['name']
        is_main_series = json_data['is_main_series']
        effect         = "\n---\n".join([effect['effect'] for effect in json_data['effect_entries']])

        sr_evolucao = pd.Series({
            "url": url,
            "evolucao_id":id,
            "name": name,
            "generation": generation,
            "is_main_series": is_main_series,
            "effect": effect
        })

        #append/concat 
        df_evolucao_full = pd.concat([df_evolucao_full,sr_evolucao.to_frame().T],ignore_index=True)

    return df_evolucao_full

In [107]:
def etapa_extracao():
    
    print("  > Base pokemons_url:\n")
    try:
        df_pokemons_url = get_base_pokemons_url()
        salva_bd(df_pokemons_url,"pokemons_url")
    except Exception as e:
        alerta(status_code = 200, 
               base = "pokemons_url", 
               etapa = "EXTRACAO", 
               erro=e)
    
    
    print("  > Base habilidades_url:\n")
    try:
        df_habilidades_url = get_base_habilidades_url()
        salva_bd(df_habilidades_url,"habilidades_url")
    except Exception as e:
        alerta(status_code = 200, 
               base = "habilidades_url", 
               etapa = "EXTRACAO", 
               erro=e)
               
    print("  > Base evolucao_url:\n")
    try:
        df_evolucao_url = get_base_evolucao_url()
        salva_bd(df_evolucao_url,"evolucao_url")
    except Exception as e:
        alerta(status_code = 200, 
               base = "evolucao_url", 
               etapa = "EXTRACAO", 
               erro=e)
        
    
    print("  > Base pokemons:\n")
    try:
        df_pokemons_full = get_base_pokemons()
        salva_bd(df_pokemons_full,"pokemons")
    except Exception as e:
        alerta(status_code = 200, 
               base = "pokemons", 
               etapa = "EXTRACAO", 
               erro=e)


    print("  > Base habilidades:\n")
    try:
        df_habilidades_full = get_base_habilidades()
        salva_bd(df_habilidades_full,"habilidades")
    except Exception as e:
        alerta(status_code = 200, 
               base = "habilidades", 
               etapa = "EXTRACAO", 
               erro=e)
               
    print("  > Base evolucao:\n")
    try:
        df_evolucao_full = get_base_evolucao()
        salva_bd(df_evolucao_full,"evolucao")
    except Exception as e:
        alerta(status_code = 200, 
               base = "evolucao", 
               etapa = "EXTRACAO", 
               erro=e)

In [108]:
def etapa_transformacao():

    try:
        #base de pokemons
        df_pokemons = carrega_bd("pokemons")
        #abrindo as linhas por habilidade
        df_pokemons['habilidades'] = df_pokemons['habilidades'].apply(eval)
        df_pokemons_open = df_pokemons.explode('habilidades')
        #buscando id das habilidades
        df_pokemons_open['ability_url'] = df_pokemons_open['habilidades'].apply(lambda row: row['ability']['url'])
        df_pokemons_open['ability_id']  = df_pokemons_open['ability_url'].apply(lambda row: int(row.split('/')[-2]))
        df_pokemons_open = df_pokemons_open.rename(columns={'name':'pokemon_name'})
        df_pokemons_open = df_pokemons_open[['pokemon_id','ability_id','pokemon_name','height','weight','base_experience']]

        #base de habilidades
        df_habilidades = carrega_bd("habilidades")
        df_habilidades = df_habilidades.rename(columns={'name':'ability_name','effect':'ability_effect'})
        df_habilidades = df_habilidades[['ability_id','ability_name','ability_effect']]

        #join da base pokemons e habilidades
        df_pokemons_habilidades = df_pokemons_open.merge(df_habilidades,on=['ability_id'],how='left')
 
        #salva resultado final
        salva_bd(df_pokemons_habilidades,"pokemons_habilidades")
        print("pokemons_habilidades salva")
    except Exception as e:
        alerta(nivel = 1, 
               base = "pokemons_habilidades", 
               etapa = "TRANSFORMACAO", 
               erro=e)

In [ ]:
etapa_extracao()